# 深度学习计算

我们将简要概括深度学习计算的各个重要组成部分，如模型构造、参数的访问和初始化等，自定义层，读取、存储和使用GPU。通过本章的学习，我们将能够深入了解模型实现和计算的各个细节，并为在之后章节实现更复杂模型打下坚实的基础。
## 模型构造
回顾一下在“多层感知机的简洁实现”一节中含单隐藏层的多层感知机的实现方法。我们首先构造Sequential实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他 节中也使用了Sequential类构造模型。这里我们介绍另外一种**基于Block类的模型构造方法**：它让模型构造更加灵活。
### 继承Block类来构造模型
Block类是nn模块里提供的一个**模型构造类**，我们可以继承它来定义我们想要的模型。下面继承Block类构造本节开头提到的多层感知机。这里定义的MLP类重载了Block类的**__init__函数和forward函数**。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [1]:
from mxnet import nd
from mxnet.gluon import nn

class MLP(nn.Block):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')  # 隐藏层
        self.output = nn.Dense(10)  # 输出层

    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        return self.output(self.hidden(x))

以上的MLP类中无须定义反向传播函数。系统将**通过自动求梯度而自动生成反向传播所需的backward函数**。

我们可以实例化MLP类得到模型变量net。下面的代码初始化net并传入输入数据X做一次前向计算。其中，net(X)会调用MLP继承自Block类的__call__函数，这个函数将调用MLP类定义的forward函数来完成前向计算。

In [2]:
X = nd.random.uniform(shape=(2, 20))
net = MLP()
net.initialize()
net(X)


[[ 0.09543005  0.04614332 -0.00286655 -0.07790347 -0.05130241  0.02942038
   0.08696643 -0.01907929 -0.04122177  0.05088575]
 [ 0.07692869  0.03099706  0.00856576 -0.04467199 -0.06926839  0.09132432
   0.06786593 -0.06187843 -0.03436673  0.04234695]]
<NDArray 2x10 @cpu(0)>

### Sequential类继承自Block类
我们刚刚提到，Block类是一个通用的部件。事实上，Sequential类继承自Block类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是**Sequential类的目的**：它提供add函数来逐一添加串联的Block子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。
下面我们实现一个与Sequential类有相同功能的MySequential类。这或许可以帮助读者更加清晰地理解Sequential类的工作机制。

In [3]:
class MySequential(nn.Block):
    def __init__(self, **kwargs):
        super(MySequential, self).__init__(**kwargs)

    def add(self, block):
        # block是一个Block子类实例，假设它有一个独一无二的名字。我们将它保存在Block类的
        # 成员变量_children里，其类型是OrderedDict。当MySequential实例调用
        # initialize函数时，系统会自动对_children里所有成员初始化
        self._children[block.name] = block

    def forward(self, x):
        # OrderedDict保证会按照成员添加时的顺序遍历成员
        for block in self._children.values():
            x = block(x)
        return x

用MySequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。

In [4]:
net = MySequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()
net(X)# 前向计算


[[ 0.00362228  0.00633331  0.03201144 -0.01369376  0.1033645  -0.03508018
  -0.00032163 -0.01676023  0.0697863   0.0130331 ]
 [ 0.03871717  0.02608213  0.03544961 -0.02521311  0.11005436 -0.0143066
  -0.03052466 -0.03852826  0.06321152  0.0038594 ]]
<NDArray 2x10 @cpu(0)>

### 构造复杂的模型
虽然Sequential类可以使模型构造更加简单，且不需要定义forward函数，但**直接继承Block类可以极大地拓展模型构造的灵活性**。下面我们构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们**通过get_constant函数创建训练中不被迭代的参数，即常数参数**。在前向计算中，除了使用创建的常数参数外，我们还使用NDArray的函数和Python的控制流，并多次调用相同的层。

In [5]:
class FancyMLP(nn.Block):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        # 使用get_constant创建的随机权重参数不会在训练中被迭代（即常数参数）
        self.rand_weight = self.params.get_constant(
            'rand_weight', nd.random.uniform(shape=(20, 20)))
        self.dense = nn.Dense(20, activation='relu')

    def forward(self, x):
        x = self.dense(x)
        # 使用创建的常数参数，以及NDArray的relu函数和dot函数
        x = nd.relu(nd.dot(x, self.rand_weight.data()) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.dense(x)
        # 控制流，这里我们需要调用asscalar函数来返回标量进行比较
        while x.norm().asscalar() > 1:
            x /= 2
        if x.norm().asscalar() < 0.8:
            x *= 10
        return x.sum()

In [6]:
net = FancyMLP()
net.initialize()
net(X)


[18.571953]
<NDArray 1 @cpu(0)>

因为FancyMLP和Sequential类都是Block类的子类，所以我们可以嵌套调用它们

In [7]:
class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.Dense(64, activation='relu'),
                     nn.Dense(32, activation='relu'))
        self.dense = nn.Dense(16, activation='relu')

    def forward(self, x):
        return self.dense(self.net(x))

net = nn.Sequential()
net.add(NestMLP(), nn.Dense(20), FancyMLP())

net.initialize()
net(X)


[24.86621]
<NDArray 1 @cpu(0)>

### 小结
可以通过继承Block类来构造模型。

Sequential类继承自Block类。

虽然Sequential类可以使模型构造更加简单，但直接继承Block类可以极大地拓展模型构造的灵活性。

## 模型参数的访问、初始化和共享
在“线性回归的简洁实现”一节中，我们通过init模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解**如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数**。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们从MXNet中导入了init模块，它包含了多种模型初始化方法。

In [8]:
from mxnet import init, nd
from mxnet.gluon import nn

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'))
net.add(nn.Dense(10))
net.initialize()  # 使用默认初始化方式

X = nd.random.uniform(shape=(2, 20))
Y = net(X)  # 前向计算

### 访问模型参数
对于使用Sequential类构造的神经网络，我们可以**通过方括号[]来访问网络的任一层**。回忆一下上一节中提到的Sequential类与Block类的继承关系。对于Sequential实例中含模型参数的层，我们可以**通过Block类的params属性来访问该层包含的所有参数**。下面，访问多层感知机net中隐藏层的所有参数。索引0表示隐藏层为Sequential实例最先添加的层。

In [9]:
net[0].params, type(net[0].params)

(dense10_ (
   Parameter dense10_weight (shape=(256, 20), dtype=float32)
   Parameter dense10_bias (shape=(256,), dtype=float32)
 ),
 mxnet.gluon.parameter.ParameterDict)

可以看到，我们得到了一个由参数名称映射到参数实例的字典（类型为ParameterDict类）。其中权重参数的名称为dense0_weight，它由net[0]的名称（dense0_）和自己的变量名（weight）组成。而且可以看到，该参数的形状为(256, 20)，且数据类型为32位浮点数（float32）。为了访问特定参数，我们既可以**通过名字来访问字典里的元素，也可以直接使用它的变量名**。下面两种方法是等价的，但通常后者 net[0].weight的代码可读性更好。

In [10]:
#只运行本小节1.2得代码
#net[0].params['dense0_weight']
net[0].weight

Parameter dense10_weight (shape=(256, 20), dtype=float32)

Gluon里参数类型为**Parameter类，它包含参数和梯度的数值**，可以分别通过**data函数和grad函数**来访问。因为我们随机初始化了权重，所以权重参数是一个由随机数组成的形状为(256, 20)的NDArray。

权重梯度的形状和权重的形状一样。因为我们还没有进行反向传播计算，所以梯度的值全为0。

In [11]:
net[0].weight.data(),net[0].weight.grad()

(
 [[-0.06046963  0.00624272 -0.03472826 ... -0.01759475  0.0686483
   -0.06360765]
  [-0.01273243 -0.02659053 -0.04718638 ...  0.02570021  0.02275064
   -0.0166979 ]
  [-0.03555115  0.01875034  0.02322027 ...  0.06564643  0.04601197
   -0.01915742]
  ...
  [ 0.03173313  0.01789995  0.02519771 ... -0.06176154 -0.03986754
   -0.04898471]
  [ 0.00564718  0.04665586 -0.00028374 ...  0.05332779  0.02100175
   -0.06427249]
  [ 0.0438781   0.05357236  0.02753124 ...  0.04084889 -0.01963295
    0.05668835]]
 <NDArray 256x20 @cpu(0)>,
 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 256x20 @cpu(0)>)

类似地，我们可以访问其他层的参数，如输出层的偏差值

In [12]:
net[1].bias.data()


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 10 @cpu(0)>

最后，我们可以使用**collect_params函数来获取net变量所有嵌套（例如通过add函数嵌套）的层所包含的所有参数**。它返回的同样是一个由参数名称到参数实例的字典。

In [13]:
net.collect_params()

sequential2_ (
  Parameter dense10_weight (shape=(256, 20), dtype=float32)
  Parameter dense10_bias (shape=(256,), dtype=float32)
  Parameter dense11_weight (shape=(10, 256), dtype=float32)
  Parameter dense11_bias (shape=(10,), dtype=float32)
)

这个函数可以通过正则表达式来匹配参数名，从而筛选需要的参数。

In [14]:
net.collect_params('.*weight')

sequential2_ (
  Parameter dense10_weight (shape=(256, 20), dtype=float32)
  Parameter dense11_weight (shape=(10, 256), dtype=float32)
)

### 初始化模型参数
我们在“数值稳定性和模型初始化”一节中描述了模型的默认初始化方法：权重参数元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。MXNet的**init模块**里提供了多种预设的初始化方法。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

In [17]:
# 非首次对模型初始化需要指定force_reinit为真
net.initialize(init=init.Normal(sigma=0.01), force_reinit=True)
net[0].weight.data()[0],net[0].bias.data()

(
 [-0.00166278 -0.01142846 -0.00730912 -0.00015643  0.00202428 -0.00790555
  -0.00881476  0.01258389  0.01822245  0.00600523 -0.01679807  0.00744363
   0.01300585 -0.01169925  0.02193714 -0.00242079  0.00471824 -0.01063402
  -0.01189927 -0.01247581]
 <NDArray 20 @cpu(0)>,
 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [18]:
#使用常数来初始化权重参数。
net.initialize(init=init.Constant(1), force_reinit=True)
net[0].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 20 @cpu(0)>

如果只想**对某个特定参数进行初始化**，我们可以调用Parameter类的initialize函数，它与Block类提供的initialize函数的使用方法一致。下例中我们对隐藏层的权重使用Xavier随机初始化方法。

In [19]:
net[0].weight.initialize(init=init.Xavier(), force_reinit=True)
net[0].weight.data()[0]


[ 0.13972151  0.13994855 -0.09956521  0.11547077 -0.06167766  0.05925046
 -0.09442322 -0.01746386 -0.04555789 -0.0863137  -0.00587973  0.04864542
  0.0065393   0.07486808  0.10427272  0.03069079  0.11484194 -0.00129254
 -0.08253687 -0.11263339]
<NDArray 20 @cpu(0)>

### 自定义初始化方法
有时候我们需要的初始化方法并没有在init模块中提供。这时，可以实现一个Initializer类的子类，从而能够像使用其他初始化方法那样使用它。通常，我们只需要实现_init_weight这个函数，并将其传入的NDArray修改成初始化的结果。在下面的例子里，我们令权重有一半概率初始化为0，有另一半概率初始化为
[−10,−5]和[5,10]两个区间里均匀分布的随机数。

In [20]:
class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        data[:] = nd.random.uniform(low=-10, high=10, shape=data.shape)
        data *= data.abs() >= 5

net.initialize(MyInit(), force_reinit=True)
net[0].weight.data()[0]

Init dense10_weight (256, 20)
Init dense11_weight (10, 256)



[ 0.         0.         8.445971   0.        -0.        -0.
  7.4766426  0.         6.679632  -0.        -5.7232933 -0.
  5.424509  -0.        -9.756577  -9.358888  -0.        -8.7801
 -5.4086514  6.9863834]
<NDArray 20 @cpu(0)>

还可以通过Parameter类的set_data函数来直接改写模型参数。例如，在下例中我们将隐藏层参数在现有的基础上加1。

In [21]:
net[0].weight.set_data(net[0].weight.data() + 1)
net[0].weight.data()[0]


[ 1.         1.         9.445971   1.         1.         1.
  8.476643   1.         7.679632   1.        -4.7232933  1.
  6.424509   1.        -8.756577  -8.358888   1.        -7.7801
 -4.4086514  7.9863834]
<NDArray 20 @cpu(0)>

### 共享模型参数
在有些情况下，我们希望在多个层之间共享模型参数。“模型构造”一节介绍了如何在Block类的forward函数里多次调用同一个层来计算。这里再介绍另外一种方法，它在构造层的时候指定使用特定的参数。如果不同层使用同一份参数，那么它们在前向计算和反向传播时都会共享相同的参数。在下面的例子里，我们让模型的**第二隐藏层（shared变量）和第三隐藏层共享模型参数**。

In [23]:
net = nn.Sequential()
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

X = nd.random.uniform(shape=(2, 20))
net(X)

net[1].weight.data()[0] == net[2].weight.data()[0]


[1. 1. 1. 1. 1. 1. 1. 1.]
<NDArray 8 @cpu(0)>

## 模型参数的延后初始化
如果做了上一节练习，你会发现模型net在调用初始化函数initialize之后、在做前向计算net(X)之前时，权重参数的形状中出现了0。虽然直觉上initialize完成了所有参数初始化过程，然而这在Gluon中却是不一定的。我们在本节中详细讨论这个话题。
### 延后初始化
也许读者早就注意到了，在之前使用Gluon创建的全连接层都没有指定输入个数。例如，在上一节使用的多层感知机net里，我们创建的隐藏层仅仅指定了输出大小为256。当调用initialize函数时，由于隐藏层输入个数依然未知，系统也无法得知该层权重参数的形状。只有在当我们将形状是(2, 20)的输入X传进网络做前向计算net(X)时，系统才推断出该层的权重参数形状为(256, 20)。因此，这时候我们才能真正开始初始化参数。
让我们使用上一节中定义的MyInit类来演示这一过程。我们创建多层感知机，并使用MyInit实例来初始化模型参数。

In [24]:
from mxnet import init, nd
from mxnet.gluon import nn

class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        # 实际的初始化逻辑在此省略了

net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'),
        nn.Dense(10))

net.initialize(init=MyInit())

虽然MyInit被调用时会打印模型参数的相关信息，但上面的initialize函数执行完并未打印任何信息。由此可见，调用initialize函数时并没有真正初始化参数。下面我们定义输入并执行一次前向计算。

In [25]:
X = nd.random.uniform(shape=(2, 20))
Y = net(X)

Init dense20_weight (256, 20)
Init dense21_weight (10, 256)


这时候，有关模型参数的信息被打印出来。在根据输入X做前向计算时，系统能够根据输入的形状自动推断出所有层的权重参数的形状。系统在创建这些参数之后，调用MyInit实例对它们进行初始化，然后才进行前向计算。
当然，这个初始化只会在第一次前向计算时被调用。之后我们再运行前向计算net(X)时则不会重新初始化，因此不会再次产生MyInit实例的输出。

In [26]:
Y = net(X)

系统将真正的参数初始化延后到获得足够信息时才执行的行为叫作**延后初始化**（deferred initialization）。它可以让模型的创建更加简单：只需要定义每个层的输出大小，而不用人工推测它们的输入个数。这对于之后将介绍的定义多达数十甚至数百层的网络来说尤其方便。

然而，任何事物都有两面性。正如本节开头提到的那样，延后初始化也可能会带来一定的困惑。在第一次前向计算之前，我们无法直接操作模型参数，例如无法使用data函数和set_data函数来获取和修改参数。因此，我们经常会额外做一次前向计算来迫使参数被真正地初始化。

### 避免延后初始化
如果系统在调用initialize函数时能够知道所有参数的形状，那么延后初始化就不会发生。我们在这里分别介绍两种这样的情况。
第一种情况是我们要对已初始化的模型重新初始化时。因为参数形状不会发生变化，所以系统能够立即进行重新初始化。

In [27]:
net.initialize(init=MyInit(), force_reinit=True)

Init dense20_weight (256, 20)
Init dense21_weight (10, 256)


第二种情况是我们在创建层的时候指定了它的输入个数，使系统不需要额外的信息来推测参数形状。下例中我们通过**in_units**来指定每个全连接层的输入个数，使初始化能够在initialize函数被调用时立即发生。

In [28]:
net = nn.Sequential()
net.add(nn.Dense(256, in_units=20, activation='relu'))
net.add(nn.Dense(10, in_units=256))

net.initialize(init=MyInit())

Init dense22_weight (256, 20)
Init dense23_weight (10, 256)


## 自定义层
深度学习的一个**魅力在于神经网络中各式各样的层**，例如全连接层和后面章节中将要介绍的卷积层、池化层与循环层。虽然Gluon提供了大量常用的层，但有时候我们依然希望自定义层。本节将介绍如何**使用NDArray来自定义一个Gluon的层**，从而可以被重复调用。
### 不含模型参数的自定义层
我们先介绍如何定义一个不含模型参数的自定义层。事实上，这和“模型构造”一节中介绍的使用Block类构造模型类似。下面的CenteredLayer类通过**继承Block类**自定义了一个将输入减掉均值后输出的层，并将层的计算定义在了forward函数里。这个层里不含模型参数。

In [29]:
from mxnet import gluon, nd
from mxnet.gluon import nn

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)

    def forward(self, x):
        return x - x.mean()

In [30]:
##实例化这个层，然后做前向计算。
layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

我们也可以用它来构造更复杂的模型。

In [31]:
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

下面打印自定义层各个输出的均值。因为均值是浮点数，所以它的值是一个很接近0的数。

In [32]:
net.initialize()
y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

-9.0221874e-10

### 含模型参数的自定义层
我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。
“模型参数的访问、初始化和共享”一节分别介绍了Parameter类和ParameterDict类。在自定义含模型参数的层时，我们可以利用Block类自带的ParameterDict类型的**成员变量params**。它是一个由字符串类型的参数名字映射到Parameter类型的模型参数的字典。我们可以通过get函数从ParameterDict创建Parameter实例。

In [33]:
params = gluon.ParameterDict()
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

尝试实现一个含权重参数和偏差参数的全连接层。它使用ReLU函数作为激活函数。其中in_units和units分别代表输入个数和输出个数。

In [34]:
class MyDense(nn.Block):
    # units为该层的输出个数，in_units为该层的输入个数
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [35]:
#实例化MyDense类并访问它的模型参数
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

直接使用自定义层做前向计算。

In [36]:
dense.initialize()
dense(nd.random.uniform(shape=(2, 5)))


[[0.00787069 0.         0.        ]
 [0.01778084 0.         0.        ]]
<NDArray 2x3 @cpu(0)>

使用自定义层构造模型。它和Gluon的其他层在使用上很类似。

In [37]:
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.02510271]
 [0.01257453]]
<NDArray 2x1 @cpu(0)>

## 读取和存储
到目前为止，我们介绍了如何处理数据以及如何构建、训练和测试深度学习模型。然而在实际中，我们有时需要**把训练好的模型部署到很多不同的设备**。在这种情况下，我们可以把内存中训练好的模型参数存储在硬盘上供后续读取使用。

### 读写NDArray
我们可以直接使用**save函数和load函数**分别存储和读取NDArray。下面的例子创建了NDArray变量x，并将其存在文件名同为x的文件里。将数据从存储的文件读回内存。

In [40]:
from mxnet import nd
from mxnet.gluon import nn

x = nd.ones(3)
nd.save('x', x)

x2 = nd.load('x')
x2

[
 [1. 1. 1.]
 <NDArray 3 @cpu(0)>]

还可以存储一列NDArray并读回内存。

In [41]:
y = nd.zeros(4)
nd.save('xy', [x, y])
x2, y2 = nd.load('xy')
(x2, y2)

(
 [1. 1. 1.]
 <NDArray 3 @cpu(0)>,
 
 [0. 0. 0. 0.]
 <NDArray 4 @cpu(0)>)

甚至可以存储并读取一个从字符串映射到NDArray的字典。

In [42]:
mydict = {'x': x, 'y': y}
nd.save('mydict', mydict)
mydict2 = nd.load('mydict')
mydict2

{'x': 
 [1. 1. 1.]
 <NDArray 3 @cpu(0)>,
 'y': 
 [0. 0. 0. 0.]
 <NDArray 4 @cpu(0)>}

### 读写Gluon模型的参数
除NDArray以外，我们还可以**读写Gluon模型的参数**。Gluon的Block类提供了**save_parameters函数和load_parameters函数**来读写模型参数。为了演示方便，我们先创建一个多层感知机，并将其初始化。回忆“模型参数的延后初始化”一节，由于延后初始化，我们需要先运行一次前向计算才能实际初始化模型参数。

In [44]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')
        self.output = nn.Dense(10)

    def forward(self, x):
        return self.output(self.hidden(x))

net = MLP()
net.initialize()
X = nd.random.uniform(shape=(2, 20))
Y = net(X)

filename = 'mlp.params'
net.save_parameters(filename)

In [45]:
net2 = MLP()
net2.load_parameters(filename)

因为这两个实例都有同样的模型参数，那么对同一个输入X的计算结果将会是一样的。我们来验证一下

In [46]:
Y2 = net2(X)
Y2 == Y


[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
<NDArray 2x10 @cpu(0)>